# Customer Segmentation for Online Retail Using RFM Analysis and Clustering

## Problem Statement
Retailers collect large volumes of transactional data but often struggle to translate this data into actionable marketing strategies.  Treating all customers the same leads to inefficient marketing spend, poor personalization, and missed revenue opportunities. The goal of this project is to use historical transaction data from an online retail store to segment customers into meaningful groups based on their purchasing behavior.

Using the Online Retail dataset, which contains detailed transaction records including customer IDs, purchase dates, quantities, and transaction values, this project aims to answer the following business questions:

- Can customers be grouped into distinct segments based on their purchasing behavior?
- Who are the most valuable customers (high spenders, frequent buyers)?
- Which customers are at risk of churn?
- Which segments should receive targeted promotions versus loyalty rewards?

To solve this, the project will engineer behavioral features using RFM analysis (Recency, Frequency, Monetary value) and apply unsupervised machine learning techniques such as K-Means clustering to group customers into interpretable segments. The results will be evaluated using clustering metrics (e.g., silhouette score) and visualized to provide clear business recommendations.

## Project Objectives
- Clean and preprocess transaction-level retail data
- Engineer RFM features for each customer
- Apply clustering algorithms to segment customers
- Interpret each segment with business meaning (e.g., loyal customers, bargain hunters, at-risk customers)
- Provide data-driven marketing recommendations for each segment

## 1. Import + Settings

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

## 2. Load the Data Set

In [42]:
df_retail = pd.read_csv("../data/OnlineRetail.csv", encoding="ISO-8859-1")

## 3. Initial Data Analysis

### 3.1 Basic Structure

In [43]:
df_retail.shape

(541909, 8)

### 3.2 Data Overview

In [44]:
df_retail.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### 3.3 Column Data Types

In [45]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [46]:
df_retail["InvoiceDate"] = pd.to_datetime(df_retail["InvoiceDate"])

### 3.4 Summary Statistics

In [47]:
df_retail.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


### 3.5 Check for Missing Values

In [48]:
df_retail.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Since some data do not have customerID, we can remove this from our data set. This is because we do not have a identifier for such products

In [49]:
df_retail = df_retail.dropna(subset=["CustomerID"]).copy()
df_retail.shape

(406829, 8)

### 3.6 Check for Duplicated Rows

In [50]:
df_retail.duplicated().sum()

5225

Lets remove the duplicated rows.

In [51]:
df_retail.drop_duplicates(inplace=True)
df_retail.shape

(401604, 8)

### 3.7 Check unique values of every column

In [52]:
df_retail.nunique().sort_values(ascending=False)

InvoiceNo      22190
InvoiceDate    20460
CustomerID      4372
Description     3896
StockCode       3684
UnitPrice        620
Quantity         436
Country           37
dtype: int64

### 3.8 Sanity Check 1: Remove Cancelled Orders

Invoice No that starts with 'C' are cancelled orders. We will remove them from the dataset.

In [53]:
df_retail = df_retail[~df_retail["InvoiceNo"].astype(str).str.startswith("C")].copy()
df_retail.shape

(392732, 8)

### 3.9 Sanity Chcek 2: Remove Invalid Quantity and Price

Remove non-positive quantity/price (returns, errors)


In [54]:
df_retail = df_retail[(df_retail["Quantity"] > 0) & (df_retail["UnitPrice"] > 0)].copy()
df_retail.shape

(392692, 8)

### 3.10 Create Revenue Column

In [55]:
df_retail['Revenue'] = df_retail['Quantity'] * df_retail['UnitPrice']
df_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


### 3.11 Define Reference Date for Recency

In [56]:
reference_date = df_retail["InvoiceDate"].max() + pd.Timedelta(days=1)
reference_date

Timestamp('2011-12-10 12:50:00')

In [57]:
rfm = (
    df_retail.groupby("CustomerID")
      .agg(
          Recency=("InvoiceDate", lambda x: (reference_date - x.max()).days),
          Frequency=("InvoiceNo", "nunique"),
          Monetary=("Revenue", "sum")
      )
      .reset_index()
)

rfm.head()
rfm.shape

(4338, 4)

### 3.12 Min Max Checks

In [58]:
print("Customers:", rfm.shape[0])
print("Min/Max Recency:", rfm["Recency"].min(), rfm["Recency"].max())
print("Min/Max Frequency:", rfm["Frequency"].min(), rfm["Frequency"].max())
print("Min/Max Monetary:", rfm["Monetary"].min(), rfm["Monetary"].max())

Customers: 4338
Min/Max Recency: 1 374
Min/Max Frequency: 1 209
Min/Max Monetary: 3.75 280206.02


## 4. Save to CSV

In [59]:
rfm.to_csv("../data/OnlineRetail_RFM_Data.csv", index=False)